<a href="https://colab.research.google.com/github/hersan19/mcpr2024/blob/main/AG_MCPR2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MCPR 2024**

Tutorial: Neuroevolución

Código: Algoritmo genético para el problema de las 8 reinas

# Algoritmo Genético
Problema de las 8 reinas

Representación planteada:

Permutaciones, la posición representa la columna y el valor del 0 al 7 representa la fila en la que se posiciona la reina.

In [ ]:
import random
import copy
import matplotlib.pyplot as plt

ejemplo = [0, 1, 2, 3, 4, 5, 6, 7]

In [ ]:
# La función decodifica hace la transición de la representación
# genotípica a la fenotípica
def decodifica (lista):
  ceros = [[0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0]]
  for con in range(8):
    ceros[lista[con]][con] = 1
  return ceros

# Función de aptitud

Conteo de ataques en un individuo

In [ ]:
# Cuenta cuántos elementos repetidos hay en una lista
# Se tomará ese conteo para el número de ataques en el tablero
def contrep (lista):
  copia = lista.copy()
  conteo = 0
  while len(copia) > 0:
    k = copia[0]
    if k not in copia[1:]:
      copia.pop(0)
    else:
      conta = 1
      for elem in copia[1:]:
        if elem == k:
          conta += 1
      conteo += conta
      while conta > 0:
        copia.remove(k)
        conta = conta - 1
  return conteo

# Calcula el número de ataques en las diagonales
def aptitud(indi, cont):
  diagasc = []
  diagdes = []
  for col in range(8):
    diagasc.append(indi[col] + col)
    diagdes.append(indi[col] - col)
  ataques = contrep(diagasc) + contrep(diagdes)
  cont += 1
  return ataques, cont

# Población de soluciones

Generar la población inicial.

In [ ]:
# Genera soluciones aleatoriamente para conformar la población
def pobini(cuantos, cont):
  poblado = []
  for miembro in range(cuantos):
    base = [0, 1, 2, 3, 4, 5, 6, 7]
    indiv = []
    while len(base) > 0:
      elemento = random.choice(base)
      indiv.append(elemento)
      base.remove(elemento)
    valu, cont = aptitud(indiv, cont)
    indiv_eval = [indiv, valu]
    poblado.append(indiv_eval)
  return poblado, cont

# Mecanismo de selección de padres

Torneo: seleccionar aleatoriamente 5 individuos e incluir los mejores dos como padres.

In [ ]:
def sele_pad (poblac):
  sel_5 = random.choices(poblac, k=5)
  sel_5.sort(key=lambda x: x[1])
  best_2 = [sel_5[0], sel_5[1]]
  return best_2

# Operadores de variación

Cruza y mutación

In [ ]:
# Determina un punto de corte aleatorio y posteriormente genera a
# descendientes, los completa si es necesario.
def cruza(pad1, pad2):
  cpad1 = copy.deepcopy(pad1[0])
  cpad2 = copy.deepcopy(pad2[0])
  corte = random.randint(0,7)
  descen1 = cpad1[0:corte]
  for elm1 in cpad2[corte:]:
    if elm1 not in descen1:
      descen1.append(elm1)
  descen2 = cpad2[0:corte]
  for elm2 in cpad1[corte:]:
    if elm2 not in descen2:
      descen2.append(elm2)
  # Completar si incompletos
  if len(descen1) < 8:
    ref = [0, 1, 2, 3, 4, 5, 6, 7]
    for mie1 in descen1:
      ref.remove(mie1)
    for i1 in range(8 - len(descen1)):
      anade = random.choice(ref)
      descen1.append(anade)
      ref.remove(anade)
  if len(descen2) < 8:
    ref = [0, 1, 2, 3, 4, 5, 6, 7]
    for mie2 in descen2:
      ref.remove(mie2)
    for i2 in range(8 - len(descen2)):
      anade = random.choice(ref)
      descen2.append(anade)
      ref.remove(anade)
  descends = [descen1, descen2]
  return descends

# Determina dos posiciones aleatorias diferentes e intercambia
# sus valores.
def mutacion (son):
  sonc = copy.deepcopy(son)
  pos1 = random.randint(0, 7)
  pos2 = random.randint(0, 7)
  while pos1 == pos2:
    pos2 = random.randint(0, 7)
  a = sonc[pos1]
  b = sonc[pos2]
  son[pos2] = a
  son[pos1] = b

# Procedimiento de cruza y mutación
def variacion (pads, p_cruz, p_muta, cont):
  hij_evals = []
  if p_cruz > random.random():
    hijos = cruza(pads[0], pads[1])
    for hij in hijos:
      if p_muta > random.random():
        mutacion(hij)
      valu, cont = aptitud(hij, cont)
      hijo_eval = [hij, valu]
      hij_evals.append(hijo_eval)
  return hij_evals, cont

# Mecanismo de reemplazo

Selección de los mejores y descarte

In [ ]:
# Ordena y va descartando al último elemento hasta tener
# el tamaño de población planteado.
def reemplazo (pobla, descen):
  control = len(pobla)
  pobla.extend(descen)
  pobla.sort(key=lambda x: x[1])
  while len(pobla) > control:
    pobla.pop(-1)

# Algoritmo completo

Implementación con las funciones elaboradas

In [ ]:
# Parámetros:
#   tpob = tamaño de la población
#   nevs = número de evaluaciones máximo
#   pcr = procentaje de cruza
#   pmu = porcentaje de mutación
#   display: 1 para mostrar la mejor solución de cada generación
#     con 0 el display se apaga.
def evo8reinas(tpob, nevs, pcr, pmu, display):
  control = 0
  poblacion, control = pobini(tpob, control)
  reg_sol = []
  genera = 0
  while control < nevs and poblacion[0][1] > 0:
    padres = sele_pad(poblacion)
    hijos, control = variacion(padres, pcr, pmu, control)
    reemplazo(poblacion, hijos)
    reg_sol.append(poblacion[0][1])
    if display == 1:
      print("\nGeneración: ", genera)
      print("Mejor solución: \n")
      print(poblacion[0][0])
      print("\nDecodificación: \n")
      for lista in decodifica(poblacion[0][0]):
        print(lista)
      print("\nCantidad de ataques: {}\n".format(poblacion[0][1]))
    genera += 1
  solu = poblacion[0]
  if display == 1:
    print("-------------------------------------------------------\n")
  print("RESULTADOS FINALES:")
  if solu[1] == 0:
    print("\nLa ejecución fue exitosa.")
    print("\nCantidad de generaciones: {}".format(genera))
    print("Evaluaciones requeridas: ", control)
    print("Solución encontrada:\n")
    print(solu[0])
    print("\nDecodificación: \n")
    for lista2 in decodifica(solu[0]):
      print(lista2)
  else:
    print("\nLa ejecución NO fue exitosa.")
    print("\nMenor cantidad de ataques: ", solu[8])
    print("Tablero:\n")
    print(solu[0][0])
    print("\nDecodificación: \n")
    for lista2 in decodifica(solu[0]):
      print(lista2)
  return reg_sol, solu, control

In [ ]:
# Tamaño de población:
poblac = 100
# Número máximo de evaluaciones:
numeval = 10000
# Porcentaje de cruza (de 0 a 100):
pcruza = 100
# Porcentaje de mutación (de 0 a 100):
pmuta = 80
# Desplegar los detalles de cada generación en la ejecución.
# sí = 1, no = 0.
displayon = 0


evolucion, soluc, evals = evo8reinas(poblac, numeval, pcruza/100, pmuta/100, displayon)

print("")
ejex = [n for n in range(1, len(evolucion) + 1)]
plt.figure(figsize=(8,6))
plt.style.use("Solarize_Light2")
plt.plot(ejex, evolucion, marker='', linewidth=4.0, label="Best individual")
plt.legend()
plt.title("Gráfica de convergencia")
plt.xlabel("Generación")
plt.ylabel("Aptitud del mejor individuo")
plt.ylim(bottom=0)
plt.show()